<a href="https://colab.research.google.com/github/MichaelSeitz98/Seminararbeit_Kleinkinderkennung/blob/main/hilfsskript_fuer_posttraining_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Google Drive mounten für Zugriff auf Datensatz und zum Speichern der trainierten Modelle



In [ ]:
# Verknüpfung mit meiner Google Drive herstellen

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Sicherstellen, dass train_labels.csv dort vorhanden sind (in keinem Unterornder)

datensatz_folder = '/content/drive/MyDrive/DS2_POOLG_80_20_20'

## Installation der Tensorflow OD API

Muss bei jeder neuen Laufzeit neu installiert werden - auch für die Inferenzanwendung.

In [ ]:
# Allgemeines Tensorflow installieren 
!pip install tensorflow=="2.6.0"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 458.4 MB 28 kB/s 
     |████████████████████████████████| 132 kB 50.6 MB/s 
     |████████████████████████████████| 14.9 MB 37.2 MB/s 
  Created wheel for clang: filename=clang-5.0-py3-none-any.whl size=30694 sha256=859a35183be3259e1f320a83693060592f09013e0fea473d244a0816d7f8bc5f
  Stored in directory: /root/.cache/pip/wheels/f1/60/77/22b9b5887bd47801796a856f47650d9789c74dc3161a26d608
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4849 sha256=52f24a8007b65984fd475ec676e2a83bc515939fe080f247fc7cbaeaca741b3e
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=72348 sha256=169ea88b923bc6b51cdccca73b37c9fe9ca7fa388feb18f9d55e4b92a78e6489
  Stored in directory: /root/.cache/pip/wheels/5f/fd

In [ ]:
# Tensorflow Modelle laden (wenn noch nicht vorhanden)
import pathlib
import os
 
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3632, done.
remote: Counting objects: 100% (3632/3632), done.
remote: Compressing objects: 100% (3019/3019), done.
remote: Total 3632 (delta 962), reused 1521 (delta 558), pack-reused 0
Receiving objects: 100% (3632/3632), 47.11 MiB | 24.51 MiB/s, done.
Resolving deltas: 100% (962/962), done.


In [ ]:
# Hier die Anpassungen in model_liv_v2.py vornehmen 

#Zeile 1136

# import regex as re
# import glob
# def natural_sort(l): 
#   convert = lambda text: int(text) if text.isdigit() else text.lower() 
#   alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
#   return sorted(l, key = alphanum_key)
# for latest_checkpoint in natural_sort(list(set(map(lambda n: n[:n.index(".")], glob.glob(f"{checkpoint_dir}/ckpt-*.*"))))):


#https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a 

In [ ]:
# Installation der Tensorflow Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
# Test, ob Installation erfolgreich

!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

## Vorbereitung für Evaluation 



In [ ]:
training_folder = '/content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100'


In [ ]:
%cd {training_folder}

/content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100


In [ ]:
# Dateipfade aller fürs Training relevanten Dateien: train.record, test.record, labelmap.pbtxt, + dann noch: config des Modelles, Alle aus dem Trainings-Ordner ziehen

train_record_path = training_folder + '/train.record'
validation_record_path =  training_folder + '/validation.record'
labelmap_path =  training_folder + '/labelmap.pbtxt'

print (f'Diese Dateien müssen vorhanden sein in: {training_folder}\n')
print (f'train_record_path: {train_record_path} \nvalidation_record_path: {validation_record_path}\nlabelmap_path: {labelmap_path}')

Diese Dateien müssen vorhanden sein in: /content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100

train_record_path: /content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/train.record 
validation_record_path: /content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/validation.record
labelmap_path: /content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/labelmap.pbtxt


## Modellauswahl und Trainingsparameter festlegen...

Modellauswahl aus: (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [ ]:
config_path = '/content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/ssd_efficientdet_d0_512x512_coco17_tpu-8.config' 
print (f'Config path gesetzt auf: {config_path}')

Config path gesetzt auf: /content/drive/MyDrive/training01_effDetD0_ds1024Erw_8_20000_100/ssd_efficientdet_d0_512x512_coco17_tpu-8.config


## Ab hier: alles vorhanden für Evaluation.

In [ ]:
# Erzeugt final Evaluation metrics z.B. mAP / iou precisions -> auch unabhängig von Training anwendbar --> mit Modifikationen auch verlauf darstellbar

eval_folder=f'{training_folder}/eval_neu'
!python /content/models/research/object_detection/model_main_tf2.py --model_dir={eval_folder} --pipeline_config_path={config_path} --checkpoint_dir={training_folder}

2022-11-29 18:52:03.192052: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 18:52:04.123272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-29 18:52:04.123394: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-29 18:52:04.123414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF